In [ ]:
# load and prepare planet dataset and save to file
import tensorflow as tensorflow
from os import listdir
from numpy import zeros
from numpy import asarray
from numpy import savez_compressed
from pandas import read_csv
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
# create a mapping of tags to integers given the loaded mapping file
def create_tag_mapping(mapping_csv):
	# dict that maps labels to integers, and the reverse
	labels_map = {'upright':0, 'rotated_right':1, 'upside_down':2, 'rotated_left':3}
	inv_labels_map = {0:'upright', 1:'rotated_right', 2:'upside_down', 3:'rotated_left'}
	return labels_map, inv_labels_map

# create a mapping of filename to a list of tags
def create_file_mapping(mapping_csv):
	mapping = dict()
	for i in range(len(mapping_csv)):
		name, tags = mapping_csv['fn'][i], mapping_csv['label'][i]
		mapping[name] = tags.split(' ')
	return mapping
 
# create a one hot encoding for one list of tags
def one_hot_encode(tags, mapping):
	# create empty vector
	encoding = zeros(len(mapping), dtype='uint8')
	# mark 1 for each tag in the vector
	for tag in tags:
		encoding[mapping[tag]] = 1
	return encoding
 
# load all images into memory
def load_dataset(path, file_mapping, tag_mapping):
	photos, targets = list(), list()
	# enumerate files in the directory
	for filename in listdir(folder):
		# load image
		photo = load_img(path + filename, target_size=(128,128))
		# convert to numpy array
		photo = img_to_array(photo, dtype='uint8')
		# get tags
		tags = file_mapping[filename]
		# one hot encode tags
		target = one_hot_encode(tags, tag_mapping)
		# store
		photos.append(photo)
		targets.append(target)
	X = asarray(photos, dtype='uint8')
	y = asarray(targets, dtype='uint8')
	return X, y
 
# load the mapping file
filename = 'train.truth.csv'
mapping_csv = read_csv(filename)
# create a mapping of tags to integers
tag_mapping, _ = create_tag_mapping(mapping_csv)
# create a mapping of filenames to tag lists
file_mapping = create_file_mapping(mapping_csv)
# load the jpeg images
folder = 'train/'
X, y = load_dataset(folder, file_mapping, tag_mapping)
print(X.shape, y.shape)
# save both arrays to one file in compressed format
savez_compressed('image_data.npz', X, y)

from numpy import load
data = load('image_data.npz')
X, y = data['arr_0'], data['arr_1']
print('Loaded: ', X.shape, y.shape)

In [2]:
from __future__ import print_function
from numpy import load
import tensorflow as tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import os

In [3]:
batch_size = 64
num_classes = 4
epochs = 5
num_predictions = 4
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_trained_model.h5'

# The data, split between train and test sets:

data = load('image_data.npz')
X, y = data['arr_0'], data['arr_1']
print('Loaded: ', X.shape, y.shape)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print('x_train shape:', x_train.shape)

Loaded:  (48896, 128, 128, 3) (48896, 4)
x_train shape: (34227, 128, 128, 3)


In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
#model.add(Conv2D(32, (3, 3)))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
#model.add(Conv2D(64, (3, 3)))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = tensorflow.keras.optimizers.Adam()

# Let's train the model using Adam
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

Train on 34227 samples, validate on 14669 samples
Epoch 1/5
34227/34227 [==============================] - 1223s 36ms/sample - loss: 0.6612 - accuracy: 0.7418 - val_loss: 0.2891 - val_accuracy: 0.8989
Epoch 2/5
34227/34227 [==============================] - 1292s 38ms/sample - loss: 0.2904 - accuracy: 0.8946 - val_loss: 0.2058 - val_accuracy: 0.9299
Epoch 3/5
34227/34227 [==============================] - 1233s 36ms/sample - loss: 0.2137 - accuracy: 0.9230 - val_loss: 0.1515 - val_accuracy: 0.9508
Epoch 4/5
34227/34227 [==============================] - 1262s 37ms/sample - loss: 0.1667 - accuracy: 0.9393 - val_loss: 0.1403 - val_accuracy: 0.9517
Epoch 5/5
34227/34227 [==============================] - 1562s 46ms/sample - loss: 0.1445 - accuracy: 0.9471 - val_loss: 0.1119 - val_accuracy: 0.9618


In [7]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at C:\Users\Bhavesh\Desktop\GL\DeeperSystem_NN\saved_models\keras_trained_model.h5 
14669/1 [=======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Test loss: 0.1118696972341317
Test accuracy: 0.96182424


In [2]:
import tensorflow as tensorflow
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import os

In [4]:
# image folder
folder_path = 'test/'
# path to model
model_path = 'saved_models/keras_trained_model.h5'
# dimensions of images
img_width, img_height = 128, 128

# load the trained model
model = load_model(model_path)
opt = tensorflow.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# load all images into a list
list_images = os.listdir(folder_path)
images = []
for img in list_images:
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img/255)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)

[3 0 1 ... 3 2 0]


In [7]:
print(classes[0:20])

[3 0 1 1 1 1 3 3 2 0 3 0 0 0 2 2 0 2 1 3]


In [10]:
test_pred = pd.DataFrame({'fn':list_images, 'label': classes})
test_pred['label'] = test_pred['label'].map({0:'upright', 1:'rotated_right', 2:'upside_down', 3:'rotated_left'})
test_pred.to_csv('test.preds.csv')

In [11]:
test_pred.head()

,fn,label
0,90-10184590_1979-06-16_2006.jpg,rotated_left
1,90-1019890_1931-08-10_1978.jpg,upright
2,90-10241990_1984-11-28_2007.jpg,rotated_right
3,90-102690_1966-09-09_2011.jpg,rotated_right
4,90-10303590_1983-01-26_2010.jpg,rotated_right


In [14]:
# image folder
folder_path = 'train/'
# path to model
model_path = 'saved_models/keras_trained_model.h5'
# dimensions of images
img_width, img_height = 128, 128

# load the trained model
model = load_model(model_path)
opt = tensorflow.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# load all images into a list
truth_list_images = os.listdir(folder_path)
images = []
for img in truth_list_images:
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img/255)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)

In [15]:
def merge(list1, list2): 
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))] 
    return merged_list 

pred_images = merge(truth_list_images, classes)

In [22]:
import cv2
import os

folder_path = 'C:/Users/Bhavesh/Desktop/GL/DeeperSystem_NN/train'

for image, pred_class in pred_images:
    os.chdir(folder_path)
    image = os.path.join(folder_path, image)
    img = cv2.imread(image)
    if pred_class == 1:
        rot_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif pred_class == 2:
        rot_img = cv2.rotate(img, cv2.ROTATE_180)
    elif pred_class == 3:
        rot_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    else:
        rot_img = img
    cv2.imwrite(image, rot_img)